<a href="https://colab.research.google.com/github/gmoraissc/Projetos_de_Data_Science/blob/main/Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [185]:
from google.colab import drive
drive.mount('/content/gdrive')
!pip freeze --local > /content/gdrive/My\ Drive/colab_installed.txt

Mounted at /content/gdrive


INSTALLATIONS

In [1]:
!pip install emoji, !pip install xlsxwriter, ! python -m spacy download pt_core_news_sm, !pip install spacy

     |████████████████████████████████| 174 kB 8.6 MB/s 
  Created wheel for emoji: filename=emoji-1.6.3-py3-none-any.whl size=170298 sha256=258c9f07b7af494de2db8b5a69812cff964a9016675aecd781a9c9dabf142754
  Stored in directory: /root/.cache/pip/wheels/03/8b/d7/ad579fbef83c287215c0caab60fb0ae0f30c4d7ce5f580eade
Successfully built emoji


IMPORTS

In [3]:
# imports
import pandas as pd
import numpy as np
import tweepy, nltk, json, os, csv, xlsxwriter, pytz, re, heapq, string, spacy
from pandas import ExcelWriter as ExcelWriter
from pathlib import Path
from datetime import datetime, date, timedelta
from csv import writer
from abc import ABCMeta, abstractmethod
from nltk.corpus import treebank
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [4]:
class auth_twitter(object):
  
  def __init__(self):
    pass

  def parametros_de_acesso(self, arquivo):
    
    self.__arquivo = arquivo

    with open (arquivo, 'r') as arquivo:
      texto = arquivo.readlines()[0].split(',')
    
    access_token = texto[0].split('=')[1]
    access_token_secret = texto[1].split('=')[1]
    bearer_token = texto[2].split('=')[1]
    consumer_key = texto[3].split('=')[1]
    consumer_secret = texto[4].split('=')[1]

    arquivo.close()

    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth, wait_on_rate_limit=True)
    
    return api

DATA EXCTRACTION

In [117]:
class pesquisar_tweets(object):

  def __init__(self):
    self.resultados_da_pesquisa = []
    self.historico_info_participantes = []
    self.informacoes_hist_participante = []

  def por_termo(self, query: tuple, result_type='recent', count=int(100)):
    
    """Este método faz a consulta de tweets a partir de termos. 
    Query: a query deve ser uma tupla (str) utilizando-se de operadores lógicos
    tais como OR, AND. Ex. BBB AND #REDEBBB. 
    Result_type: pode ser ou popular ou recent. O primeiro buscará os tweets
    que corresponderem a query publicados recentemente. O segundo será aqueles
    tweets mais populares. O padrão é recent.
    Count: recebe um valor inteiro para limitar o número de tweets coletados por consulta. O padrão é 100.
    Retorna os resultados da pesquisa (lista) e o tipo de resultado (popular ou recent).
    """

    global api

    self.__query = query
    self.__result_type = result_type
    self.__count = count

    for status in tweepy.Cursor(api.search,
                                q=self.__query,
                                result_type=self.__result_type,
                                count=self.__count).items():
                                
                                self.resultados_da_pesquisa.append(status)

    return self.resultados_da_pesquisa, self.__result_type
 
  def por_usuario(self, participante: str, result_type='recent', count=int(100)):
    
    """Este método faz a consulta de tweets a partir do nome de um usuário. 
    Participante: recebe o screen_name do usuário do twitter. 
    Result_type: pode ser ou popular ou recent. O primeiro buscará os tweets
    que corresponderem a query publicados recentemente. O segundo será aqueles
    tweets mais populares. O padrão é recent.
    Count: recebe um valor inteiro para limitar o número de tweets coletados por consulta. O padrão é 100.
    Retorna os resultados da pesquisa em uma lista, o tipo de resultado (popular ou recent) e o tipo de
    participante (camarote, pipoca, público ou perfil oficial).
    """

    global api, participantes_camarote, participantes_pipoca # ajustar para quando as informações de pesquisa estiverem na classe

    self.__participante = participante
    self.__result_type = result_type
    self.__count = count
    
    tipo_participante = informacoes_de_participantes().checar_tipo_participante(participante)

    for status in tweepy.Cursor(api.user_timeline,
                                screen_name=self.__participante,
                                result_type=self.__result_type,
                                exclude_replies=False).items(self.__count):
                                
                                self.resultados_da_pesquisa.append(status)

    return self.resultados_da_pesquisa, self.__result_type, tipo_participante

  def por_usuarios(self, participantes: list, result_type='recent', count=int(100)):
    
    """Este método faz a consulta de tweets a partir do nome de pelo menos um usuário. 
    Participante: recebe o screen_name dos usuários do twitter que deseja ser consultado. 
    Result_type: pode ser ou popular ou recent. O primeiro buscará os tweets
    que corresponderem a query publicados recentemente. O segundo será aqueles
    tweets mais populares. O padrão é recent.
    Count: recebe um valor inteiro para limitar o número de tweets coletados por consulta. O padrão é 100.
    Retorna uma lista com os resultados da pesquisa, o tipo de resultado (popular ou recent) e o tipo de pesquisa (única ou múltipla).
    """

    global api

    self.__participantes = participantes
    self.__result_type = result_type
    self.__count = count
    
    for participante in self.__participantes:

      for status in tweepy.Cursor(api.user_timeline,
                                  screen_name=participante,
                                  result_type=self.__result_type,
                                  exclude_replies=False).items(self.__count):
                                  
                                  self.resultados_da_pesquisa.append(status)

    return self.resultados_da_pesquisa, self.__result_type, 'Todos'

  def infos_do_usuario(self, usuario: str):
  
    """Este método faz a consulta de informações do usuário (número de seguidores (int), número de perfis que segue (int),
    número de  (int), número de tweets favoritados (int), número de tweets totais (int), verificado (booleano) e a data
    é uma informação gerada pelo método "corrigir_timezone" cujo padrão é a hora, a fim de organizar os arquivos nas devidas pastas). 
    Usuario: recebe o screen_name do usuário do twitter que deseja ser consultado.
    Retorna uma lista.
    """

    global api

    self.__user_results = api.get_user(usuario)

    extracao = corrigir_timezone.__func__(tipo='hora')
    
    participante = self.__user_results.name
    twitter = self.__user_results.screen_name
    seguidores = self.__user_results.followers_count
    seguindo = self.__user_results.friends_count
    numero_de_listados = self.__user_results.listed_count
    n_tweets_favoritados = self.__user_results.favourites_count
    num_de_tweets_totais = self.__user_results.statuses_count
    verificado = self.__user_results.verified

    self.informacoes_hist_participante.append({'participante': str(participante),
                                          'twitter': str(twitter),
                                          'n_seguidores': int(seguidores),
                                          'n_seguindo': int(seguindo),
                                          'n_de_listados': int(numero_de_listados),
                                          'n_tweets_favoritados': int(n_tweets_favoritados),
                                          'num_de_tweets_totais': int(num_de_tweets_totais),
                                          'verificado': bool(verificado),
                                          'data': extracao})
    
    return self.informacoes_hist_participante

  def historico_participante(self, participantes: list):

    """Este métood visa, a partir da busa por usuario, iterar em uma lista de usuários e fazer uma varredura por todos os participantes
    buscando as informações históricas (citadas em infos_do_usuario).
    Participantes: recebe uma lista de screen_names para iteração.
    Retorna uma lista com todas as informações coletadas.
    """

    for participante in participantes:
      _ = pesquisar_tweets().infos_do_usuario(participante)
      self.historico_info_participantes.extend(_)
    
    return self.historico_info_participantes, 'historico', 'todos'

DATA ORGANIZATION/FILE HANDLING

In [129]:
class organizar_coletas_de_tweets(object):

  def __init__(self):
    self.dicionario = {}
    self.lista = []

  def criar_dicionario(self, tweets_status, tipo):
      
    d1 = corrigir_timezone.__func__(tipo='data')

    self.__tweets_status = tweets_status
    self.__tipo = tipo

    for each_json_tweet in tweets_status:
      _ = json.dumps(each_json_tweet._json)
      
      if d1 not in self.dicionario.keys():
        tweet = {d1: [json.loads(_)]}
        self.dicionario.update(tweet)
        
      else:
        self.dicionario[d1].append(json.loads(_))
      
    return self.dicionario, self.__tipo

  def dataframe_tweets(self, dicionario, tipo_de_pesquisa, multipla=False):
    
    d1 = corrigir_timezone.__func__(tipo='data')
    
    self.__dicionario = dicionario
    self.__tipo_de_pesquisa = tipo_de_pesquisa

    for i in range(0, len(dicionario[d1])):

          tweet_id = self.__dicionario[d1][i]['id']
          text = self.__dicionario[d1][i]['text']
          favorite_count = self.__dicionario[d1][i]['favorite_count']
          retweet_count = self.__dicionario[d1][i]['retweet_count']
          created_at = self.__dicionario[d1][i]['created_at']
          hashtags = self.__dicionario[d1][i]['entities']['hashtags']
          user_mentions = self.__dicionario[d1][i]['entities']['user_mentions']
          name = self.__dicionario[d1][i]['user']['name']
          screen_name = self.__dicionario[d1][i]['user']['screen_name']

          self.lista.append({'tweet_id': str(tweet_id),
                        'text': str(text),
                        'favorite_count': int(favorite_count),
                        'retweet_count': int(retweet_count),
                        'created_at': created_at,
                        'user_mentions': user_mentions,
                        'name': name,
                        'screen_name': screen_name})
          
    self.__tweet_json_ = pd.DataFrame(self.lista, columns = 
                              ['tweet_id', 'text', 
                                'favorite_count', 'retweet_count', 
                                'created_at', 'hashtags', 
                                'user_mentions', 'name',
                                'screen_name'])
    
    if multipla == False:
      
      self.__tipo_participante = informacoes_de_participantes().checar_tipo_participante(screen_name)
    
    else:
      
      self.__tipo_participante = 'Todos'

    return self.__tweet_json_, self.__tipo_participante, self.__tipo_de_pesquisa

  def dataframe_dos_participantes(self, informacoes_hist_participante):

      self.__informacoes_hist_participante = informacoes_hist_participante

      self.__dados_participantes = pd.DataFrame(self.__informacoes_hist_participante, columns = 
                                      ['participante', 'twitter', 'n_seguidores', 
                                      'n_seguindo','n_de_listados', 'n_tweets_favoritados',
                                      'num_de_tweets_totais', 'verificado', 'data'])
      
      return self.__dados_participantes, 'historico', None
  
  @staticmethod
  def salvar_arquivo(dataframe, tipo_pesquisa, tipo_participante):
    
    data_extracao = corrigir_timezone.__func__(tipo='data')
    dir_csvs = f'/content/drive/MyDrive/01.Instagram profissional/NLP/Sentiment Analysis/BBB22/Twitter/DataFrames/'

    if tipo_pesquisa == 'historico':
           
      file_variable_ = dir_csvs + 'Participantes'
      file_name = 'historico.csv'
      file_path_ = file_variable_ + "/" + file_name
      diretorios(file_variable=file_variable_, file_path=file_path_, dataframe=dataframe)
    
    else:

      file_variable_ = dir_csvs + "/" + tipo_participante  + "/" + tipo_pesquisa + "/" + data_extracao
      file_name = tipo_participante[0:3] + "_" + tipo_pesquisa[0:3] + "_" + data_extracao + '.csv'
      file_path_ = file_variable_ + "/" + file_name
      diretorios(file_variable=file_variable_, file_path=file_path_)

  @staticmethod     
  def diretorios(file_variable, file_path, dataframe):

    if not os.path.exists(file_variable):
      os.makedirs(file_variable)
      dataframe.to_csv(file_path, index=False)

    else:
      with open(file_path, 'a') as f_object:
        writer_object = writer(f_object, delimiter=',')
        
        linhas = []
        
        for row in range(0, len(dataframe)):
          linhas.append([row for row in dataframe.iloc[row]])
          writer_object.writerow(linhas[row])

        f_object.close()
    
  @staticmethod
  def corrigir_timezone(tipo):
    
    utcmoment_naive = datetime.utcnow()
    utcmoment = utcmoment_naive.replace(tzinfo=pytz.utc)
    tz = 'America/Sao_Paulo'
    extracao = utcmoment.astimezone(pytz.timezone(tz)) - timedelta(hours=0, minutes=60)
    data_de_extracao = extracao.date().strftime("%d%m%Y")

    if tipo == 'data':
      return data_de_extracao
    
    elif tipo == 'hora':
      return extracao

PARTICIPANTS INFORMATIONS

In [156]:
class informacoes_de_participantes(object):

  def __init__(self):
    pass

  @staticmethod
  def checar_tipo_participante(usuario):
      if usuario in participantes_camarote:
        return 'Camarote'
      elif usuario in participantes_pipoca:
        return 'Pipoca'
      elif usuario in perfis_oficiais_bbb:
        return 'PerfilOficial'
      else:
        return 'Publico'
  
  #def status_do_participante -> eliminado? anjo? lider? acompanhar ao longo do tempo e comparar com o "desempenho" no twitter

TEXT OPERATIONS

In [13]:
# metodo para adicionar termos mais relevantes para o modelo *publico e por participante, exemplo, memes
#termos mais frequentes para determinado participante
class termos_de_pesquisa(object):
  
  def __init__(self):
    pass

  def termos(self):
    
    dicionario_de_termos_de_pesquisa = {'BigBrotherBrasil': {'BBB', 'BBB22', '#REDEBBB', '#BBB', '#BBB22'},
         'Participantes': {'Arthur': ['Arthur', 'Artur', 'Aguiar', 'Arthur Aguiar', '\u2747\ufe0f', '#TeamAguiar', '#TeamArthurAguiar'],
                           'Naiara Azevedo': ['Naiara', 'Azevedo', '1f4b8', '#TeamNaiara', 'Nai', 'cantora'],
                           'Pedro Scooby': ['Pedro', 'Scooby', '1f30a', '#TeamScooby', '#TimeScooby', '1f499'],
                           'Brunna Gonçalves': ['Brunna', 'Gonçalves', '1f984', '#BBBRUNNA', 'BBBrunna'],
                           'Paulo André': ['Paulo', 'André', '1f3c1', '#TeamPauloAndré', 'TeamPauloAndre'],
                           'Maria': ['Maria', '1f40d	', '#TEAMMARIA', 'TIME MARIA', 'MARICONAS'],
                           'Jade Picon': ['Jade', 'Picon', '1f32a\ufe0f', 'furacão', '#TeamJade', 'Picão', 'KdJade', 'Picões', 'Furacao'],
                           'Douglas Silva': ['Douglas', 'Silva', '1f3b2', 'dado', 'dadinho', '#TeamDouglasSilva', '1f44a\U0001f3ff'],
                           'Linn da Quebrada': ['Linn', 'Quebrada', '1f9dc\u200d\u2640\ufe0f', '1f9dc\U0001f3ff\u200d\u2640\ufe0f',
                                                '1f9dc\U0001f3fb\u200d\u2640\ufe0f', '1f9dc\U0001f3fe\u200d\u2640\ufe0f', '1f9dc\U0001f3fc\u200d\u2640\ufe0f',
                                                '1f9dc\U0001f3fd\u200d\u2640\ufe0f', '1f9dc', '1f9dc\U0001f3ff', '1f9dc\U0001f3fb', '1f9dc\U0001f3fe',
                                                '1f9dc\U0001f3fc', '1f9dc\U0001f3fd', '#TeamLinn', '#linndonas', '#LinnDonas'],
                           'Tiago Abravanel': ['Tiago', 'Abravanel', '1f43b', '#TeamAbrava', '#TeamAbravanel', '1f9f8'],
                           'Laís Caldas': ['Laís', 'Caldas', '1f462', '#TeamLais', '#TeamLaís', 'Time Lais', '1f49a'],
                           'Luciano Estevan': ['Luciano', 'Estevan', '1f981', '#TeamLucianoEstevan', 'Lu', '1f346'],
                           'Jessilane': ['Jessilane', '1f9ec', 'Jessi', '#TeamJessi', '#TimeJessi', '1f49c', 'Charmanders'],
                           'Eliezer': ['Eliezer', '1F437', 'Eli', '#TeamEli', '1F953', '1F416', '1F43D'],
                           'Eslovênia Marques': ['Eslovênia', 'Eslováquia', 'Marques', 'Eslô', '1f1f8\U0001f1ee',
                                                 'Time Eslo', 'Team Eslo', '#TimeEslô', '#TeamEslô', 'Eslovenia'],
                           'Bábara Heck': ['Bárbara', 'Heck', '1f980', '#TeamBá', 'TeamBá', 'TimeBá',
                                           'TimeBa', 'TeamBa', 'TeamBah', 'BBBah', 'Bá', 'Ba', '1f9a6'],
                           'Rodrigo Mussi': ['Rodrigo Mussi', 'Mussi', 'Rodrigo', 'TEAM MUSSI', '#TeamMussi',
                                             '#TimeMussi', '1f977\U0001f3ff', '1f977', '1f977\U0001f3fb',
                                             '1f977\U0001f3fe', '1f977\U0001f3fc', '1f977\U0001f3fd', 'ninja', 'ninjas', 'TEAM NINJA'],
                           'Natália Deodato': ['Natália', 'Deodato', '#TeamNaty', '#TimeNaty', 'Naty', 'vitilindos',
                                               '1f483', '1f483\U0001f3ff', '1f483\U0001f3fb',
                                               '1f483\U0001f3fe', '1f483\U0001f3fc', '1f483\U0001f3fd'],
                           'Vinicius': ['Vinicius', 'TeamVyni', '#TimeVyni', '#TeamVyni', '1f4a1'],
                           'Lucas Bissoli': ['Lucas', 'Bissoli', '#TimeBissoli', '#TeamBissoli', 
                                             '1f3c4\u200d\u2642\ufe0f', '1f3c4\U0001f3ff\u200d\u2642\ufe0f', 
                                             '1f3c4\U0001f3fb\u200d\u2642\ufe0f', '1f3c4\U0001f3fe\u200d\u2642\ufe0f',
                                             '1f3c4\U0001f3fc\u200d\u2642\ufe0f', '1f3c4\U0001f3fd\u200d\u2642\ufe0f',
                                             '1f3c4', '1f3c4\U0001f3ff', '1f3c4\U0001f3fb', '1f3c4\U0001f3fe',
                                             '1f3c4\U0001f3fc', '1f3c4\U0001f3fd', '1f3c4\u200d\u2640\ufe0f',
                                             '1f3c4\U0001f3ff\u200d\u2640\ufe0f', '1f3c4\U0001f3fb\u200d\u2640\ufe0f',
                                             '1f3c4\U0001f3fe\u200d\u2640\ufe0f', '1f3c4\U0001f3fc\u200d\u2640\ufe0f',
                                             '1f3c4\U0001f3fd\u200d\u2640\ufe0f']}}
    return dicionario_de_termos_de_pesquisa
    #termos_a_analisar = []

    #avaliador_de_termos()

    #return termos_de_pesquisa_BBB

    #analisador_de_termos(termos_de_pesquisa_participantes)

  def analisador_de_termos(self, termos_atuais, termos_novos):

    self.__termos_atuais = termos_atuais
    self.__termos_novos = termos_novos

    #for termo in termos_atuais:
      # if novo termo ajudar a explicar:
      # append
      # if len(termos) > limite:
      # termo_a_remover = ()
      # for termo in termos atuais atualizado:
      # termo_a_remover['explicacao'] < min_a_explicar:
      # termos_atuais.pop(termo_a_remover)

  def tendencia(self, termo_novo):

    self.termo_novo = termo_novo
    pass

  def correlacao_participantes(self):
    pass

In [14]:
dicionario_participantes = {'@Aguiarthur': 'Arthur Aguiar', 
                            '@Naiarazevedo': 'Naiara Azevedo',
                            '@PedroScooby': 'Pedro Scooby',
                            '@brunnagoncalves': 'Bruna Gonçalves',
                            '@iampauloandre': 'Paulo André',
                            '@eumaria': 'Maria',
                            '@jadepicon': 'Jade Picon',
                            '@Silva_DG': 'Douglas Silva',
                            '@linndaquebrada': 'Linn da Quebrada',
                            '@TiagoAbravanel': 'Tiago Abravanel',
                            '@Dra_laiscaldass': 'Laís Caldas',
                            '@LucianoEstevan': 'Luciano Estevan',
                            '@a_jessilane': 'Jessilane',
                            '@eusouoeli': 'Eliezer',
                            '@eslomarques': 'Eslovênia Maques',
                            '@bbaheck': 'Bábara Heck',
                            '@oficialmussi': 'Rodrigo Mussi',
                            '@oficial_deodato': 'Natália Deodato',
                            '@vyniof': 'Vinicius',
                            '@LucasBissoli_': 'Lucas Bissoli'}

participantes_camarote = ['Aguiarthur', 'Naiarazevedo', 'PedroScooby', 'brunnagoncalves', 
                          'iampauloandre', 'eumaria', 'jadepicon', 'Silva_DG', 'linndaquebrada', 'TiagoAbravanel']

participantes_pipoca = ['Dra_laiscaldass', 'LucianoEstevan', 'a_jessilane', 'Eli', 'eslomarques', 
                        'bbaheck', 'oficialmussi', 'oficial_deodato', 'vyniof', 'LucasBissoli_']

participantes_camarote_twitter = ['@Aguiarthur', '@Naiarazevedo', '@PedroScooby', '@brunnagoncalves', 
                          '@iampauloandre', '@eumaria', '@jadepicon', '@Silva_DG', '@linndaquebrada', '@TiagoAbravanel']

participantes_pipoca_twitter = ['@Dra_laiscaldass', '@LucianoEstevan', '@a_jessilane', '@eusouoeli', '@eslomarques', 
                        '@bbaheck', '@oficialmussi', '@oficial_deodato', '@vyniof', '@LucasBissoli_']

perfis_oficiais_bbb = ['bbb', 'globoplay', 'tadeuschmidt']

participantes_total = []
participantes_total.extend(participantes_pipoca)
participantes_total.extend(participantes_camarote)

TESTES

In [15]:
api = auth_twitter().parametros_de_acesso('/content/drive/MyDrive/01.Instagram profissional/NLP/Sentiment Analysis/BBB22/Twitter/auth.txt')

PESQUISAS POR NOME DE USUARIO(S)

In [130]:
pesquisar = pesquisar_tweets()
organizar = organizar_coletas_de_tweets()

In [60]:
tweets, tipo_de_pesquisa, tipo_de_participante = pesquisar.por_usuario('@PedroScooby')
dicionario, tipo_de_pesquisa = organizar.criar_dicionario(tweets, tipo_de_pesquisa)
dataframe, tipo_de_participante, tipo_de_pesquisa = organizar.dataframe_tweets(dicionario, tipo_de_pesquisa, multipla=False)
salvar_arquivo(tipo_pesquisa=tipo_de_pesquisa, dataframe=dataframe, tipo_participante=tipo_de_participante)

In [96]:
tweets, tipo_de_pesquisa, tipo_de_participante = pesquisar.por_usuario('@bbb')
dicionario, tipo_de_pesquisa = organizar.criar_dicionario(tweets, tipo_de_pesquisa)
dataframe, tipo_de_participante, tipo_de_pesquisa = organizar.dataframe_tweets(dicionario, tipo_de_pesquisa, multipla=False)
organizar.salvar_arquivo(tipo_pesquisa=tipo_de_pesquisa, dataframe=dataframe, tipo_participante=tipo_de_participante)

In [61]:
tweets, tipo_de_pesquisa, tipo_de_participante = pesquisar.por_usuarios(participantes_total)
dicionario, tipo_de_pesquisa = organizar.criar_dicionario(tweets, tipo_de_pesquisa)
dataframe, tipo_de_participante, tipo_de_pesquisa = organizar.dataframe_tweets(dicionario, tipo_de_pesquisa, multipla=True)
organizar.salvar_arquivo(tipo_pesquisa=tipo_de_pesquisa, dataframe=dataframe, tipo_participante=tipo_de_participante)

HISTÓRICO DOS PARTICIPANTES

In [121]:
tweets, tipo_de_pesquisa, tipo_de_participante = pesquisar.historico_participante(participantes_total)
dataframe, tipo_de_pesquisa, tipo_de_participante = organizar.dataframe_dos_participantes(tweets)
organizar.salvar_arquivo(dataframe=dataframe, tipo_pesquisa=tipo_de_pesquisa, tipo_participante=tipo_de_participante)

PESQUISA POR TERMO

In [122]:
termos = termos_de_pesquisa().termos()
dicionario_de_termos = termos['BigBrotherBrasil']
query = (' '.join([str(item) for item in dicionario_de_termos]).replace(' ', ' OR ')) #metodo estático para criar a query a partir de um dicionario dado
tweets, tipo_de_pesquisa = pesquisar.por_termo(query=query, result_type='popular')
dicionario, tipo_de_pesquisa = organizar.criar_dicionario(tweets, tipo_de_pesquisa)
dataframe, tipo_participante, tipo_de_pesquisa = organizar.dataframe_tweets(dicionario, tipo_de_pesquisa)
organizar.salvar_arquivo(tipo_participante=tipo_participante, tipo_pesquisa=tipo_de_pesquisa, dataframe=dataframe)

PESQUISA PERFIL OFICIAL

In [133]:
tweets, tipo_de_pesquisa, tipo_participante = pesquisar.por_usuario(participante='@bbb', result_type='popular', count=200)
dicionario, tipo_de_pesquisa = organizar.criar_dicionario(tweets, tipo_de_pesquisa)
dataframe, tipo_de_participante, tipo_de_pesquisa = organizar.dataframe_tweets(dicionario, tipo_de_pesquisa)
organizar.salvar_arquivo(tipo_participante=tipo_de_participante, tipo_pesquisa=tipo_de_pesquisa, dataframe=dataframe)

MELHORIAS

In [155]:
# Funcionalidades

# property para não precisar "herdar" as informações de tipo de pesquisa e tipo de consulta (GETTER)
# coletas automáticas/schedule!
# criar uma base de textos de todos os tweets/csvs
# rf para definir se tweet é spam de tweet ou campanha de ganhar seguidores para excluir da base
# usar texto do perfil oficial do bbb para "lider" anjo provas etc

# Analises

# em média, quantos seguidores ganha após um tweet positivo? e quantos perde após um tweet negativo?
# modelo de classificação baseado na popularidade do participante
# prob de ser eliminado no paredão de acordo com o atual nível de popularidade

NLP

In [224]:
class processamento_de_texto(object):

  def __init__(self):
    self.texto_processado = []
    self.texto_sendo_analisado = []
    self.texto_sem_pontuacao = []
    self.texto_sem_stopwords = []
    self.texto_tokenizado = []

  def remover_https_e_espacamento(self, base_de_textos):
    
    for tweet in base_de_textos:
        self.__sem_https = re.sub(r'https.+', '', str(tweet))
        self.__sem_caractere_de_espacamento = re.sub(r'\n+', ' ', self.__sem_https)
        self.texto_sendo_analisado.append(self.__sem_caractere_de_espacamento)
    
    return self.texto_sendo_analisado

  def remover_pontuacao(self, base_de_textos):
    
    for _ in base_de_textos:
      self.__s = re.sub(r'\W', ' ', _)
      self.__x = re.sub(' +', ' ', self.__s)
      self.texto_sem_pontuacao.append(self.__x)
      
    return self.texto_sem_pontuacao

  def remover_stopwords(self, base_de_textos):
    
    for _ in base_de_textos:
      self.__x = [palavra for palavra in _.split() if palavra.lower() not in nltk.corpus.stopwords.words('portuguese')]
      self.texto_sem_stopwords.append(self.__x)

    return self.texto_sem_stopwords

  def tokenizar_texto(self, base_de_textos):

    for i in range(0, len(base_de_textos)):
      
      for _ in base_de_textos[i]:
        
        self.__x = word_tokenize(_)
        self.texto_tokenizado.append(self.__x)
    
    return self.texto_tokenizado

  def preprocessar(self, tweets, column):

    self.__base_de_tweets = [row for row in tweets.iloc[:,column]]

    self.__sem_link_nem_espaco = self.remover_https_e_espacamento(self.__base_de_tweets)
    self.__sem_pontuacao = self.remover_pontuacao(self.__sem_link_nem_espaco)  
    self.__sem_stopwords = self.remover_stopwords(self.__sem_pontuacao)
    self.__tokenizado = self.tokenizar_texto(self.__sem_stopwords)
    self.texto_processado.append(self.__tokenizado)
    self.texto_processado_ = self.list_reduction(self.texto_processado)
    
    return self.texto_processado_
  
  def list_reduction(self, texto_processado):

    self.__texto_processado = texto_processado

    return [item for sublist in self.__texto_processado[0] for item in sublist]

In [160]:
df = pd.read_csv(f'/content/drive/MyDrive/01.Instagram profissional/NLP/Sentiment Analysis/BBB22/Twitter/DataFrames/PerfilOficial/popular/23012022/Per_pop_23012022.csv',
                 usecols=[1])
df.head(1)

,text
0,E vamos de música no #BBB22 🎤🎶\n\n#RedeBBB htt...


In [191]:
nlp = spacy.load(f'/content/gdrive/MyDrive/01.Instagram profissional/NLP/Sentiment Analysis/BBB22/Twitter/BaseSpacyPT/pt_core_news_sm-2.2.5/pt_core_news_sm/pt_core_news_sm-2.2.5')

In [225]:
a = processamento_de_texto().preprocessar(tweets=df, column=0)

In [227]:
type(a)

list

In [228]:
doc = nlp(a[0])